In [16]:
url = "https://www.youtube.com/watch?v=7BeeQipYT58"

In [37]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_script(url):
    try:
        video_id = url.split("=")[1]
        # YouTube 영상의 자막을 가져옴
        transcript = YouTubeTranscriptApi.get_transcript(video_id,languages=('ko','en'))
        
        return transcript
    except Exception as e:
        return f"Error: {str(e)}"


transcript = get_script("https://youtu.be/HsxWL3kDYh8?si=aGGS9vmvp6dNX8P2")
print(transcript)

Error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=aGGS9vmvp6dNX8P2! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!


In [40]:
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound, TranscriptsDisabled, VideoUnavailable

def get_script(url):
    try:
        # 유튜브 영상 ID 추출
        video_id = url.split("=")[1]
        
        # 자막을 가져옴
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
        
        return transcript
    except NoTranscriptFound:
        return "Error: 이 영상에는 자막이 없습니다."
    except TranscriptsDisabled:
        return "Error: 이 영상의 자막이 비활성화되어 있습니다."
    except VideoUnavailable:
        return "Error: 이 영상을 찾을 수 없습니다. 링크가 올바른지 확인하세요."
    except Exception as e:
        return f"Error: 알 수 없는 오류가 발생했습니다. {str(e)}"

Error: 이 영상의 자막이 비활성화되어 있습니다.


In [18]:
from pytubefix import YouTube
from pytubefix.cli import on_progress

yt = YouTube(url,on_progress_callback=on_progress)
audio = yt.streams.filter(only_audio=True).first().download(output_path='.',filename='audio.wav')

from pydub import AudioSegment

# 다운로드된 오디오 파일 로드 및 변환
input_file = "audio.wav"
output_file = "converted_audio.wav"

# 기존 파일을 로드하고 WAV 형식으로 변환
audio = AudioSegment.from_file(input_file)
audio.export(output_file, format="wav", codec="pcm_s16le")

<_io.BufferedRandom name='converted_audio.wav'>

In [20]:
import whisper
import torch

# 사용할 수 있는 장치를 확인하여 설정
if torch.cuda.is_available():  # CUDA(GPU)가 사용 가능한지 확인
    device = torch.device("cuda")
else:  # GPU가 없을 경우 CPU 사용
    device = torch.device("cpu")

def recognize_speech(file_path, language='ko'):
    # Whisper 모델 로드 (모델 크기는 tiny, base, small, medium, large 중 선택 가능)
    model = whisper.load_model("medium",device=device)  # 'base' 대신 원하는 모델을 선택하세요.

    # 오디오 파일을 Whisper로 변환
    result = model.transcribe(file_path, language=language)

    # Whisper 결과에서 각 세그먼트를 사용하여 시간 정보와 텍스트 추출
    transcript_with_timestamps = []
    for segment in result['segments']:
        start = segment['start']  # 시작 시간 (초 단위)
        end = segment['end']      # 종료 시간 (초 단위)
        text = segment['text']    # 텍스트

        # 시작 시간과 지속 시간을 계산하여 딕셔너리 형식으로 저장
        transcript_with_timestamps.append({
            'text': text.strip(),
            'start': start,
            'duration': end - start
        })

    return transcript_with_timestamps

In [21]:
text = recognize_speech("converted_audio.wav")

/Users/jinucho/miniconda3/envs/youtube/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_lo

In [2]:
import io

import librosa
import numpy as np
import torch
import whisper
from pydub import AudioSegment
from pytubefix import YouTube
from pytubefix.cli import on_progress
from youtube_transcript_api import (NoTranscriptFound, TranscriptsDisabled,
                                    VideoUnavailable, YouTubeTranscriptApi)


def get_youtube_transcript_or_audio_transcription(url):
    try:
        # 유튜브 영상 ID 추출
        video_id = url.split("=")[1]

        # 1. 자막 추출 시도
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
            return transcript
        
        except (NoTranscriptFound, TranscriptsDisabled):
            pass

        # 2. 자막이 없는 경우, 음성 추출 및 Whisper 사용
        # 유튜브 영상에서 오디오 스트림 다운로드
        yt = YouTube(url, on_progress_callback=on_progress)
        audio_stream = yt.streams.filter(only_audio=True).first()
        
        # 오디오 파일을 메모리 내에서 처리
        audio_data = io.BytesIO()
        audio_stream.stream_to_buffer(audio_data)  # 오디오 스트림을 버퍼에 저장
        audio_data.seek(0)  # 파일 포인터를 처음으로 이동

        # pydub을 사용하여 오디오 데이터를 로드하고, raw 데이터를 가져옴
        audio_segment = AudioSegment.from_file(audio_data)
        
        # 오디오 데이터를 numpy 배열로 변환
        samples = np.array(audio_segment.get_array_of_samples()).astype(np.float32)

        # Whisper가 기대하는 샘플 레이트(16000Hz)로 변환
        if audio_segment.frame_rate != 16000:
            samples = samples.reshape((-1, audio_segment.channels))  # 채널 정보 포함
            samples = librosa.resample(samples, orig_sr=audio_segment.frame_rate, target_sr=16000).flatten()

        # Whisper 모델이 기대하는 포맷으로 변환
        samples = samples / 32768.0  # int16 -> float32로 정규화

        # 3. Whisper를 사용하여 음성 인식 및 텍스트 변환
        if torch.cuda.is_available():  # GPU 사용 가능 여부 확인
            device = torch.device("cuda")
        else:  # GPU가 없을 경우 CPU 사용
            device = torch.device("cpu")

        # Whisper 모델 로드
        model = whisper.load_model("medium", device=device)  # 원하는 모델 크기 선택 가능

        # 오디오 파일 변환
        result = model.transcribe(samples, language=['ko', 'en'])

        # Whisper 결과에서 각 세그먼트를 사용하여 시간 정보와 텍스트 추출
        transcript_with_timestamps = []
        for segment in result['segments']:
            start = segment['start']  # 시작 시간 (초 단위)
            end = segment['end']      # 종료 시간 (초 단위)
            text = segment['text']    # 텍스트

            # 시작 시간과 지속 시간을 계산하여 딕셔너리 형식으로 저장
            transcript_with_timestamps.append({
                'text': text.strip(),
                'start': start,
                'duration': end - start
            })

        # 4. 변환된 텍스트 반환
        return transcript_with_timestamps
        
    except VideoUnavailable:
        return "Error: 이 영상을 찾을 수 없습니다. 링크가 올바른지 확인하세요."
    except Exception as e:
        return f"Error: 알 수 없는 오류가 발생했습니다. {str(e)}"

# 테스트용 URL
result = get_youtube_transcript_or_audio_transcription("https://youtu.be/HsxWL3kDYh8?si=aGGS9vmvp6dNX8P2")
print(result)